In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.nonparametric.api as nparam
from statsmodels.nonparametric.kernel_regression import KernelReg

from functions import *

from scipy.optimize import minimize
from multiprocess import Pool

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = pd.read_excel('data.xlsx')

data = data.loc[(data['drug_name'] == 'Gemcitabine') & (data['cell_line'] == 'OCUBM')][['Drug_concentration (µM)',
                                                                               'viability1', 'viability2', 'viability3', 'viability4', 'viability5','viability6']]
X = np.log10(sum([data['Drug_concentration (µM)'].values.tolist()]*6,[]))
y= sum([data.iloc[:,i].values.tolist() for i in range(1,7)],[])

df = pd.DataFrame({'y':y,'x':X })

df = df.sort_values(by=['y'])

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
y = df.y
X = df.x

#b0=1, b1=1 90 % obs 
#b0=0, b=1 85 %
b0 = 0
b1 = 4

In [4]:
df.min()

y    0.069340
x   -5.346787
dtype: float64

In [5]:
n = len(df)
omega = bernoulli.rvs(pi(y,b0,b1), size=n)

In [6]:
sum(omega)/len(df)

0.4791666666666667

In [7]:
df['omega'] = omega

In [8]:
df_cc = df[df.omega==1]

In [9]:
df_cc.min()

y        0.101910
x       -5.346787
omega    1.000000
dtype: float64

In [10]:
model = KernelReg(endog=[df.y],exog=[df.x],reg_type='lc',var_type='c', bw='cv_ls')


In [11]:
model = KernelReg(endog=[df.y],exog=[df.x],reg_type='lc',var_type='c', bw='cv_ls')

model_cc = KernelReg(endog=[df_cc.y],exog=[df_cc.x],reg_type='lc',var_type='c', bw='cv_ls')


In [12]:
model.bw

array([0.19538834])

In [13]:
def r_parallel(h):
    #print(1)
    def fn(j):
        df1 = df.drop([j],axis=0)
        y_hat = df.omega[j]*nw_mis(h,df.x[j],df1.x,df1.y,pi(df1.y,b0,b1),df1.omega)
        return (df.omega[j]/pi(df.y[j],b0,b1))*(df.y[j]-y_hat)**2

    with Pool(8) as p:
        return np.mean(p.map(fn, range(len(df))))

In [14]:
%%time
# r_parallel(0.1)
h_ipw = minimize(r_parallel, 0.7).x

ZeroDivisionError: float division by zero

In [15]:
h_df = pd.DataFrame({'$h_{ipw}$':h_ipw,'$h_{full}$': model.bw, '$h_{cc}$': model_cc.bw})

NameError: name 'h_ipw' is not defined

In [ ]:
h_df

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20, 10))

plt.plot(df['x'],df['y'],'o',mfc='none',color='black')
plt.plot(df_cc['x'],df_cc['y'],'o',color='black')


#Full model 
ax.plot(np.linspace(-5.3, -2, n),model.fit(np.linspace(-5.3, -2, n))[0] ,color= 'black', lw=2)

#CC estimator 
ax.plot(np.linspace(-5.3, -2, n),model_cc.fit(np.linspace(-5.3, -2, n))[0] ,color= 'blue', lw=2)

#IPW estimator 
ax.plot(np.linspace(-5.3, -2, n),[nw_mis(h_ipw,t,X,y,pi(y,b0,b1),omega) for t in np.linspace(-5.3, -2, n)],color= 'red', lw=2)



In [ ]:
df['res_sqr'] = (df.y -  model.fit(df.x)[0])**2

df_cc['res_sqr'] = (df_cc.y - model_cc.fit(df_cc.x)[0])**2

In [ ]:
#Full residual-based
model_var = KernelReg(endog=[df.res_sqr],exog=[df.x],reg_type='lc',var_type='c', bw='cv_ls')

model_var_cc = KernelReg(endog=[df_cc.res_sqr],exog=[df_cc.x],reg_type='lc',var_type='c', bw='cv_ls')

#Direct method 
h_fix = 0.8

model_dir_vn = KernelReg(endog=[(df.y)**2],exog=[df.x],reg_type='lc',var_type='c', bw='{h}') 
model_dir_mn = KernelReg(endog=[df.y],exog=[df.x],reg_type='lc',var_type='c', bw='{h}') 

#Differences 

#Residual

In [ ]:
df_diff = df[['x','y']]

In [ ]:
df_diff['y'] =df_diff[['y']].diff().apply(lambda y:(y**2)/2)

In [ ]:
model_diff = KernelReg(endog=[df_diff.y],exog=[df_diff.x],reg_type='ll',var_type='c', bw='{h}') 

In [ ]:
model_dir_mn = KernelReg(endog=[df_diff.y],exog=[df_diff.x],reg_type='ll',var_type='c', bw='{h}') 

In [ ]:
def r_parallel_res(h):
    #print(1)
    def fn(j):
        df1 = df.drop([j],axis=0)
        y_hat = df.res_sqr[j]*nw_mis(h,df.x[j],df1.x,df1.res_sqr,pi(df1.y,b0,b1),df1.omega)
        #return (df.omega[j]/pi(df.res_sqr[j],b0,b1))*(df.res_sqr[j]-y_hat)**2
        return (df.omega[j]/pi(df.res_sqr[j],b0,b1))*(df.res_sqr[j]-y_hat)**2

    with Pool(8) as p:
        return np.mean(p.map(fn, range(len(df))))

In [ ]:
%%time
# r_parallel(0.1)
h_ipw_var = minimize(r_parallel, 0.1).x

In [ ]:
h_ipw_var

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15, 10))

plt.plot(df['x'],df['res_sqr'],'o',mfc='none',color='black')

plt.plot(df_cc['x'],df_cc['res_sqr'],'o',color='grey')

#Full model 
#Direct 
ax.plot(np.linspace(-5.3, -2, n),model_var.fit(np.linspace(-5.3, -2, n))[0] ,color= 'black', lw=2)


#IPW
ax.plot(np.linspace(-5.3, -2, n),[sigma_mis_res(h_ipw_var[0],t,X,df.res_sqr,pi(y,b0,b1),omega)**2 for t in np.linspace(-5.3, -2, n)],'-g',color= 'red', lw=2)

#Complete-case 
ax.plot(np.linspace(-5.3, -2, n),model_var_cc.fit(np.linspace(-5.3, -2, n))[0] ,color= 'blue', lw=1,linestyle='dashed')

ax.set_ylim(0,0.0175)
